In [1]:
import os
os.chdir('/Documents/CBA_Stats')

from cbastats import DBHelper
import datetime
import pandas as pd

# Connect to Database

In [2]:
mongodbio =DBHelper.MongoDBHelper() 
client = mongodbio.create_connection(DBHelper.MONGODB_USERNAME,DBHelper.MONGODB_PWD,DBHelper.MONGODB_ENDPOINT)

db = client['cbaStats']

existing database ['cbaStats', 'admin', 'local']


In [3]:
db.list_collection_names()

['cbaGamesStaging',
 'uniqueBoxScores',
 'seasonTeamStats',
 'seasonPlayerStats',
 'cbaGames']

In [4]:
coll_cbaGames = db['cbaGames']

# pull games stats

In [5]:
season = '20-21'

In [6]:
all_data = mongodbio.select_records(coll_cbaGames,filter={'详细统计':{"$ne":''},'赛季':season})

# convert to a pandas dataframe
all_box_scores = []
for game in all_data:
    for row in game['详细统计']:
        all_box_scores.append(row)
        
all_box_scores=pd.DataFrame(all_box_scores)
all_box_scores['篮板'] = all_box_scores['进攻篮板']+all_box_scores['防守篮板']

In [7]:
all_box_scores

,球员_link,球员,号码,出场时间,首发,进攻篮板,防守篮板,助攻,犯规,抢断,...,球队,对手,地点,2分中,2分投,3分中,3分投,罚球中,罚球投,篮板
0,http://cba.sports.sina.com.cn/cba/player/show/...,黄荣奇,1,40,1,0,6,6,5,0,...,江苏,青岛,诸暨,2,6,3,6,9,9,6
1,http://cba.sports.sina.com.cn/cba/player/show/...,史鸿飞,4,31,1,0,5,3,3,4,...,江苏,青岛,诸暨,6,6,1,5,3,4,5
2,http://cba.sports.sina.com.cn/cba/player/show/...,刘宇轩,21,30,1,0,2,1,6,1,...,江苏,青岛,诸暨,2,6,0,2,2,2,2
3,http://cba.sports.sina.com.cn/cba/player/show/...,吴羽佳,16,28,1,4,2,2,3,1,...,江苏,青岛,诸暨,0,2,2,3,0,0,6
4,http://cba.sports.sina.com.cn/cba/player/show/...,吴冠希,11,3,1,0,0,0,0,0,...,江苏,青岛,诸暨,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9929,http://cba.sports.sina.com.cn/cba/player/show/...,王睿,21,0,0,0,0,0,0,0,...,同曦,天津,诸暨,0,0,0,0,0,0,0
9930,http://cba.sports.sina.com.cn/cba/player/show/...,刘羽楠,18,0,0,0,0,0,0,0,...,同曦,天津,诸暨,0,0,0,0,0,0,0
9931,http://cba.sports.sina.com.cn/cba/player/show/...,白杰,5,0,0,0,0,0,0,0,...,同曦,天津,诸暨,0,0,0,0,0,0,0
9932,http://cba.sports.sina.com.cn/cba/player/show/...,曹飞,29,0,0,0,0,0,0,0,...,同曦,天津,诸暨,0,0,0,0,0,0,0


In [8]:
coll_cbaAllGamesBoxScore = db['cbaAllGamesBoxScore']

In [9]:
mongodbio.insert_records(coll_cbaAllGamesBoxScore,all_box_scores.to_dict('record'))

cbaAllGamesBoxScore operation acknowledged!
9934 records were inserted into cbaAllGamesBoxScore.


# Get Unique BoxScore

In [11]:
all_box_scores.columns

Index(['球员_link', '球员', '号码', '出场时间', '首发', '进攻篮板', '防守篮板', '助攻', '犯规', '抢断',
       '失误', '盖帽', '扣篮', '被侵', '快攻', '得分', '轮次', '日期', '赛季', 'GameID_Sina',
       '球队ID', '对手ID', '球队', '对手', '地点', '2分中', '2分投', '3分中', '3分投', '罚球中',
       '罚球投', '篮板'],
      dtype='object')

In [12]:
simple_boxscores = all_box_scores[['篮板', '助攻','抢断','盖帽','得分']].copy()

In [13]:
uniq_simple_boxscores = simple_boxscores.drop_duplicates().sort_values(by='得分',ascending=False).reset_index(drop=True)

In [14]:
uniq_simple_boxscores

,篮板,助攻,抢断,盖帽,得分
0,8,12,5,0,60
1,14,6,0,1,55
2,11,10,4,0,55
3,3,19,3,0,54
4,12,7,6,0,53
...,...,...,...,...,...
3386,5,3,0,0,0
3387,3,3,0,0,0
3388,1,0,1,1,0
3389,0,0,0,3,0


In [16]:
coll_uniqueBoxScores= db['uniqueBoxScores']

In [110]:
mongodbio.insert_records(coll_uniqueBoxScores,records = uniq_simple_boxscores.to_dict('record'))

uniqueBoxScores operation acknowledged!
3391 records were inserted into uniqueBoxScores.


# Pull back uniqe boxscore (so that it has _id)

In [17]:
unique_boxscores = mongodbio.select_records(coll_uniqueBoxScores,{})
# unique_boxscore = pd.DataFrame(unique_boxscore)
unique_boxscores[:5]

[{'_id': ObjectId('6000ab376bb8c4991083b878'),
  '篮板': 8,
  '助攻': 12,
  '抢断': 5,
  '盖帽': 0,
  '得分': 60},
 {'_id': ObjectId('6000ab376bb8c4991083b879'),
  '篮板': 14,
  '助攻': 6,
  '抢断': 0,
  '盖帽': 1,
  '得分': 55},
 {'_id': ObjectId('6000ab376bb8c4991083b87a'),
  '篮板': 11,
  '助攻': 10,
  '抢断': 4,
  '盖帽': 0,
  '得分': 55},
 {'_id': ObjectId('6000ab376bb8c4991083b87b'),
  '篮板': 3,
  '助攻': 19,
  '抢断': 3,
  '盖帽': 0,
  '得分': 54},
 {'_id': ObjectId('6000ab376bb8c4991083b87c'),
  '篮板': 12,
  '助攻': 7,
  '抢断': 6,
  '盖帽': 0,
  '得分': 53}]

In [18]:
def return_nonid_dict(dictionary:dict)->dict:
    new_dict={}
    for key,value in dictionary.items():
        if key=='_id':
            continue
        new_dict[key]=value
    return new_dict

In [19]:
return_nonid_dict(unique_boxscores[4])

{'篮板': 12, '助攻': 7, '抢断': 6, '盖帽': 0, '得分': 53}

In [20]:
unique_boxscores[4]

{'_id': ObjectId('6000ab376bb8c4991083b87c'),
 '篮板': 12,
 '助攻': 7,
 '抢断': 6,
 '盖帽': 0,
 '得分': 53}

In [22]:
# takes a few minutes
occurences=[]
for boxscore in unique_boxscores:
    occurences.append({'boxscore_id':boxscore['_id'],
                       'occurence':mongodbio.select_records(coll_cbaAllGamesBoxScore,return_nonid_dict(boxscore))})

In [23]:
len(occurencesences)

3391

In [36]:
# update the stats in mongo
results=[]
for occurence in occurences:
    results.append(coll_uniqueBoxScores.update_one({'_id':occurence['boxscore_id']},
                                                   {"$set":{"事件":occurence['occurence']}}))

In [37]:
len(results)

3391